In [ ]:
%matplotlib inline

from skewt import SkewT

S=SkewT.Sounding("OTX_09082016.txt")
parcel=S.get_parcel(method='ml')
print parcel
print S.get_cape(*parcel)
draw()

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
from skewt import SkewT

df_snd = pd.read_csv('RAP_KEPH_130601_21z_F00.csv')

# Retrieve surface temperature
base_tmp = df_snd.loc[0]['TMP']
base_hgt = df_snd.loc[0]['HGT']

# Add the DALR

df_snd['DALR'] = base_tmp - ((df_snd.HGT-base_hgt)/1000)*9.8

# Virtual Temperature
df_snd['VIRTT'] = (df_snd.TMP+273.15)/(1 - 0.379*(6.11*np.power(((7.5*df_snd.DPT)/(237.7+df_snd.DPT)),10))/df_snd.level)-273.15

# Thermal Index
df_snd['TI'] = df_snd.TMP - df_snd.DALR

hght = df_snd[['HGT']].as_matrix().flatten()
pres = df_snd[['level']].as_matrix().flatten()
temp = df_snd[['TMP']].as_matrix().flatten()
dwpt = df_snd[['DPT']].as_matrix().flatten()
sknt = df_snd[['WSPD']].as_matrix().flatten()
drct = df_snd[['WDIR']].as_matrix().flatten()

mydata=dict(zip(('hght','pres','temp','dwpt','sknt', 'drct'),(hght, pres, temp, dwpt, sknt, drct)))
S=SkewT.Sounding(soundingdata=mydata)
S.plot_skewt(color='r')


In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
from skewt import SkewT

# Dewpoint calculation adapted from ...
def dew_point(df_snd):
    
    df_snd['DPT_B'] = df_snd.TMP_C.apply(lambda x: 17.368 if x > 0 else 17.966)
    df_snd['DPT_C'] = df_snd.TMP_C.apply(lambda x: 238.88 if x > 0 else 247.15)

    pa = df_snd.RH / 100. * np.exp(df_snd.DPT_B * df_snd.TMP_C / (df_snd.DPT_C + df_snd.TMP_C))
    df_snd['DEWP_C'] = df_snd.DPT_C * np.log(pa) / (df_snd.DPT_B - np.log(pa))

def calc_hgt(df_snd, p):
    upper_hgt, upper_level = df_snd.loc[df['level'] <= p].iloc[0][['HGT','level']]
    lower_hgt, lower_level = df_snd.loc[df['level'] >= p].iloc[-1][['HGT', 'level']]
    print upper_level, p, lower_level
    lvls = range(int(upper_level),int(lower_level)+1)
    hghts = np.empty(len(lvls))
    hghts[:] = np.NAN
    hghts[0] = upper_hgt
    hghts[-1] = lower_hgt
    df_hght = pd.DataFrame({'level': lvls, 'HGT': hghts}).interpolate()
    hgt, level = df_hght.loc[df_hght['level'] == int(p)].iloc[0][['HGT','level']]
    return hgt


df = pd.read_csv('5efc712e-0656-4003-b981-88a2348f3756.csv')

# Geopotential Height
df_hgt = df.loc[df['paramId'] == 156][0:37]
df_hgt = df_hgt.rename(columns={'value':'HGT'}).drop('paramId', 1)

# Temperature
df_tmp = df.loc[df['paramId'] == 130][0:37]
df_tmp = df_tmp.rename(columns={'value':'TMP_K'}).drop('paramId', 1)
 
# Relative Humidity
df_rh = df.loc[df['paramId'] == 157][0:37]
df_rh = df_rh.rename(columns={'value':'RH'}).drop('paramId', 1)

# U component of wind
df_uw = df.loc[df['paramId'] == 131][0:37]
df_uw = df_uw.rename(columns={'value':'W_U'}).drop('paramId', 1)

# V component of wind
df_vw = df.loc[df['paramId'] == 132][0:37]
df_vw = df_vw.rename(columns={'value':'W_V'}).drop('paramId', 1)

# Ground Temperature
df_gtmp = df.loc[df['paramId'] == 167]

dfs = [df_hgt, df_tmp, df_rh, df_uw, df_vw]

df_snd = reduce(lambda left,right: pd.merge(left,right,on='level'), dfs)

# direction = np.arctan2(u,v)*(180./np.pi)
# speed = (u**2 + v**2)**(0.5)

# Wind Speed
df_snd['W_SPD_MS'] = (df_snd.W_U**2 + df_snd.W_V**2)**(0.5)
df_snd['W_SPD_KTS'] = df_snd.W_SPD_MS * 1.94384

# Wind Direction
df_snd['W_DIR'] = np.arctan2(df_snd.W_U, df_snd.W_V) * (180./np.pi)

# Temperature in Celcius
df_snd['TMP_C'] = df_snd.TMP_K - 273.15

# Dewpoint Temperature
dew_point(df_snd)

# Retrieve surface temperature
base_tmp = df_snd.loc[0]['TMP_C']
base_hgt = df_snd.loc[0]['HGT']

# Add the DALR

df_snd['DALR'] = base_tmp - ((df_snd.HGT-base_hgt)/1000)*9.8

# Virtual Temperature
df_snd['VIRTT'] = (df_snd.TMP_K)/(1 - 0.379*(6.11*np.power(((7.5*df_snd.DEWP_C)/(237.7+df_snd.DEWP_C)),10))/df_snd.level)-273.15

# Thermal Index
df_snd['TI'] = df_snd.TMP_C - df_snd.DALR

hght = df_snd[['HGT']].as_matrix().flatten()
pres = df_snd[['level']].as_matrix().flatten()
temp = df_snd[['TMP_C']].as_matrix().flatten()
dwpt = df_snd[['DEWP_C']].as_matrix().flatten()
sknt = df_snd[['W_DIR']].as_matrix().flatten()
drct = df_snd[['W_SPD_KTS']].as_matrix().flatten()

mydata=dict(zip(('hght','pres','temp','dwpt','sknt', 'drct'),(hght, pres, temp, dwpt, sknt, drct)))
S=SkewT.Sounding(soundingdata=mydata)
parcel=S.get_parcel(method='mu')
Plcl, Plfc, P_el, CAPE, CIN = S.get_cape(*parcel)

Hlcl = calc_hgt(df_snd, Plcl)
Hlfc = calc_hgt(df_snd, Plfc)
H_el = calc_hgt(df_snd, P_el)
print Plcl, Hlcl, Plfc, Hlfc, P_el, H_el, CAPE, CIN 
S.plot_skewt(title='Thermal Plot')


In [ ]:
lvls = range(700,726)
hghts = np.empty(len(lvls))
hghts[:] = np.NAN
hghts[0] = 2827.12
hghts[-1] = 3106.7
df_hght = pd.DataFrame({'level': lvls, 'HGT': hghts}).interpolate()
print df_hght
df_lookup = df_hght.loc[df_hght['level'] == 710]
print df_lookup
hgt, level = df_lookup.iloc[0][['HGT','level']]
print level, hgt

In [ ]:
import os
import sys
import gdal
import math
from gdalconst import GA_ReadOnly

def open_raster():
    """
    This functions opens the raster file for processing
    """
    try:
        raster = gdal.Open('SRTM/srtm_12_03.tif', GA_ReadOnly)
    except RuntimeError, exception:
        print 'Unable to open '+'srtm_12_03.tif'
        print exception
        sys.exit(1)
    return raster

def retrieve_band(longitude, latitude):
    """
    This function will take in the given coordinates and return the
    elevation(band) NOTE: this only takes in Mercator value does not
    work with WGS84
    x - coordinates for the x axis or the longitude that users defined
    y - coordinates for the y axis or the latitude that user defined
    """
    if -180.0 > longitude > 180.0 or -90 > latitude > 90:
        return NULL_VALUE
    else:
        raster = open_raster()
        transform = raster.GetGeoTransform()
        x_offset = int((longitude - transform[0]) / transform[1])
        y_offset = int((latitude - transform[3]) / transform[5])
        band = raster.GetRasterBand(1)
        data = band.ReadAsArray(x_offset, y_offset, 1, 1)
        return data[0]
    
print retrieve_band(-121.7968, 48.0579)